## Описание бизнеса:
- Онлайн-школа SkillFactory
- С 2016 года SkillFactory разработала и запустила 25 курсов и специализаций, обучение по которым прошли уже более 9000 студентов, и стала лидером на рынке онлайн-образования в сегменте Data Science.

## Задача:
- Увеличение среднего чека, за счет рекомендаций других курсов платформы, основываясь на выбраном пользователем курсе. 

## План работы:
1. Построить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы.
2. Составить итоговую таблицу с рекомендациями.

## Сбор и обработка данных:
- Рассмотрим датасеты в metabase
- Составим SQl запросы для импорта необходимых данных
- Проведем анализ с помощью Python

Импортиреум необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import warnings
warnings.filterwarnings('ignore')

Выявим кол-во клиентов, приобретавших курсы 

In [2]:
def getUsersCount():
    query = '''select 
    count(distinct user_id)
    from final.carts c left join final.cart_items it on c.id = it.cart_id
    where resource_type='Course' and state='successful'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

user_count = pd.DataFrame(getUsersCount())
user_count

,count
0,49006


Выявим кол-во курсов на плотформе 

In [3]:
def getCourses():
    query = '''select count(distinct resource_id)
    from final.cart_items
    where resource_type='Course'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

course_count = pd.DataFrame(getCourses())
course_count

,count
0,127


Выявим среднее число купленных курсов на одного пользователя

In [4]:
def getMeanCourse():
    query = '''with a as
    (
    select 
    user_id,
    count(resource_type) as courses
    from final.carts c left join final.cart_items it  on c.id = it.cart_id
    
    group by 1
    )

    select 
    avg(courses)
    from a
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

mean_course = pd.DataFrame(getMeanCourse())
mean_course

,avg
0,1.4268084379537108


Выявим кол-во клиентов, приобретавших более одного курса 

In [5]:
def getUsersCountMoreThenOne():
    query = '''with a as
    (
    select user_id, 
    count(distinct resource_id) as courses
    from final.carts c left join final.cart_items it on c.id = it.cart_id
    where resource_type='Course' and state='successful' 
    group by 1
    )

    select 
    count(user_id)
    from a
    where courses > 1
    
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

user_count_more_then_one = pd.DataFrame(getUsersCountMoreThenOne())
user_count_more_then_one

,count
0,12656


Создадим новый,общий датасет, где в первой колонке будет id пользователя, а во второй id курса, который он приобрел. 

In [6]:
def getUsersAndCourses():
    query = '''with a as
    (
    select user_id, 
    count(distinct resource_id) as courses
    from final.carts c left join final.cart_items it on c.id = it.cart_id
    where resource_type='Course' and state='successful'
    group by 1
    )

    select 
    a.user_id, 
    it.resource_id
    from a join final.carts c on a.user_id = c.user_id join final.cart_items it on it.cart_id = c.id
    where a.courses > 1 and it.resource_type='Course' and c.state='successful'
    order by 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

users_and_courses = pd.DataFrame(getUsersAndCourses())
users_and_courses.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


Преобразуем получившийся датафрейм, сгруппируем по user_id и преобразуем столбец resource_id в списики из курсов 

In [7]:
new_users_and_courses = users_and_courses.groupby(['user_id'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
new_users_and_courses.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


Создадим список в который загрузим пары из курсов. 
Затем посчитаем кол-во каждой пары.

In [8]:
import itertools

pairs = []
for resource in new_users_and_courses['resource_id']: #создаем цикл для прохода по столбцу resource_id 
    for resource_pairs in itertools.combinations(resource,2): #выделяем пары и загружаем в новый список 
        pairs.append(resource_pairs)
        
from collections import Counter

common_pairs = Counter(pairs)
common_pairs

Counter({(516, 1099): 25,
         (356, 357): 100,
         (356, 1125): 44,
         (357, 1125): 52,
         (553, 1147): 16,
         (361, 1138): 40,
         (1125, 1140): 1,
         (551, 745): 138,
         (553, 745): 212,
         (551, 1138): 14,
         (553, 568): 83,
         (514, 517): 10,
         (514, 566): 138,
         (517, 566): 21,
         (363, 511): 99,
         (363, 562): 77,
         (363, 563): 33,
         (511, 562): 55,
         (511, 563): 19,
         (562, 563): 53,
         (568, 745): 102,
         (509, 553): 48,
         (509, 745): 59,
         (1125, 1144): 22,
         (509, 568): 46,
         (509, 672): 5,
         (568, 672): 4,
         (516, 552): 12,
         (356, 552): 7,
         (357, 571): 112,
         (509, 516): 35,
         (516, 568): 54,
         (513, 1141): 34,
         (571, 1125): 122,
         (551, 552): 177,
         (551, 744): 16,
         (551, 862): 8,
         (552, 744): 8,
         (552, 862): 6,
         (55

Создадим датафрейм на основе получившегося словаря 

In [9]:
common_pairs_df = pd.DataFrame.from_dict(common_pairs, orient='index').reset_index()
common_pairs_df.columns = ['courses','count']
common_pairs_df.head()

,courses,count
0,"(516, 1099)",25
1,"(356, 357)",100
2,"(356, 1125)",44
3,"(357, 1125)",52
4,"(553, 1147)",16


Разделим приобретенные пары курсов по очереднсти. Сначала первый купленный курс, затем второй 

In [10]:
sep_df = pd.concat([common_pairs_df.drop(['courses'], axis=1),common_pairs_df['courses'].apply(pd.Series)], axis=1)
sep_df.columns = ['count','1','2']
sep_df.head()

,count,1,2
0,25,516,1099
1,100,356,357
2,44,356,1125
3,52,357,1125
4,16,553,1147


Сортируем получившийся датафрейм. Выделим курсы, купленные во вторую очередь в порядке убывания.

In [11]:
sep_df = sep_df.sort_values(by=['1','count'],ascending=[True,False]).reset_index(drop=True)
sep_df = sep_df[['1','2','count']]
sep_df.head()

,1,2,count
0,356,571,103
1,356,357,100
2,356,551,48
3,356,1125,44
4,356,514,35


Сгруппируем данные по первому купленному курсу, а также передадим общий признак для дальнейшего анализа. Для этого хорошо подходит метод rank()

In [12]:
sep_df['rank'] = sep_df.groupby('1')['count'].rank(method='first', ascending=False)
sep_df

,1,2,count,rank
0,356,571,103,1.0
1,356,357,100,2.0
2,356,551,48,3.0
3,356,1125,44,4.0
4,356,514,35,5.0
...,...,...,...,...
3984,1185,1187,2,2.0
3985,1186,1187,4,1.0
3986,1186,1201,1,2.0
3987,1186,1198,1,3.0


Для рекомендательной системы будем использовать первые 2 по популярнности курса, после приобретения первого курса.

In [13]:
rank_df = sep_df[(sep_df['rank'] <= 2)]
rank_df.head()

,1,2,count,rank
0,356,571,103,1.0
1,356,357,100,2.0
74,357,571,112,1.0
75,357,1125,52,2.0
144,358,570,25,1.0


Так как большинство значений будет предельно малы, посчитаем 75% квартиль и будем отталкиваться от него 

In [14]:
rank_df['count'].describe()

count    229.000000
mean      48.069869
std       83.817913
min        1.000000
25%        5.000000
50%       17.000000
75%       52.000000
max      797.000000
Name: count, dtype: float64

Максимальное кол-во одной пары 797, а 75% всех пар находятся ниже значения 52

In [15]:
rank_df = rank_df[(rank_df['count'] > 52)]
rank_df.head()

,1,2,count,rank
0,356,571,103,1.0
1,356,357,100,2.0
74,357,571,112,1.0
347,363,511,99,1.0
348,363,562,77,2.0


Формируем новый датафрейм с 3 столбцами.
- Приобретенный курс 
- Первый по популярности после приобретенного 
- Второй по популярнсти после первого

In [16]:
pairs_pivot = rank_df.pivot_table(columns = 'rank', index = '1', values = '2', aggfunc='first')
pairs_pivot = pairs_pivot.reset_index()
pairs_pivot.columns = ['1','2','3']
pairs_pivot

,1,2,3
0,356,571.0,357.0
1,357,571.0,NaN
2,363,511.0,562.0
3,366,551.0,NaN
4,489,551.0,515.0
5,490,566.0,551.0
6,502,551.0,566.0
7,503,551.0,513.0
8,504,572.0,569.0
9,507,570.0,752.0


Уберем строки с пропущенными значениями и преобразуем тип в int

In [17]:
fin = pairs_pivot.dropna()

for x in ['1','2','3']:
    fin[x] = fin[x].astype(int)


## Итоговая таблица:
* 1 - Курс, к которому идёт рекомендация.
* 2 - Курс для рекомендации № 1 (самый популярный).
* 3 - Курс для рекомендации № 2 (второй по популярности).


In [18]:
fin.reset_index(drop=True)

,1,2,3
0,356,571,357
1,363,511,562
2,489,551,515
3,490,566,551
4,502,551,566
5,503,551,513
6,504,572,569
7,507,570,752
8,514,551,515
9,515,551,523
